In [40]:
import pandas as pd

In [41]:
df = pd.read_json("llm_traces.json")

In [42]:
print(df.columns)
df.head()

Index(['project', 'version', 'traces'], dtype='object')


,project,version,traces
0,LLM Evaluation Exercise,1,"{'trace_id': 'trace_001', 'user_query': 'Who w..."
1,LLM Evaluation Exercise,1,"{'trace_id': 'trace_002', 'user_query': 'Expla..."
2,LLM Evaluation Exercise,1,"{'trace_id': 'trace_003', 'user_query': 'Trans..."


In [43]:
df["traces"][1]

{'trace_id': 'trace_002',
 'user_query': 'Explain quantum entanglement simply.',
 'metadata': {'user_id': 'u456',
  'region': 'US',
  'timestamp': '2025-09-24T20:16:00Z'},
 'steps': [{'name': 'prompt_construction',
   'latency_ms': 50,
   'input': 'Explain quantum entanglement simply.',
   'output': 'Explain in simple words: quantum entanglement...'},
  {'name': 'llm_generation',
   'model': 'gpt-4',
   'temperature': 0.7000000000000001,
   'latency_ms': 400,
   'token_usage': {'prompt_tokens': 45, 'completion_tokens': 60},
   'output': 'Quantum entanglement is like having two dice that always show the same number...'}],
 'final_output': 'Quantum entanglement means two particles behave as one, no matter the distance.',
 'metrics': {'faithfulness': 0.92,
  'relevance': 0.9500000000000001,
  'cost_usd': 0.0038}}

In [44]:
import json

with open("llm_traces.json", "r") as f:
    data = json.load(f)

# Flatten nested JSON
df = pd.json_normalize(data)

print(df.columns)
df.head()

Index(['project', 'version', 'traces'], dtype='object')


,project,version,traces
0,LLM Evaluation Exercise,1.0,"[{'trace_id': 'trace_001', 'user_query': 'Who ..."


In [45]:
import json
import pandas as pd

with open("llm_traces.json", "r") as f:
    data = json.load(f)

# Flatten the `traces` list while keeping project + version
df = pd.json_normalize(
    data,
    record_path="traces",
    meta=["project", "version"]
)

print(df.columns)
df.head()

Index(['trace_id', 'user_query', 'steps', 'final_output', 'metadata.user_id',
       'metadata.region', 'metadata.timestamp', 'metrics.faithfulness',
       'metrics.relevance', 'metrics.cost_usd', 'final_output.fr',
       'final_output.es', 'final_output.it', 'project', 'version'],
      dtype='object')


,trace_id,user_query,steps,final_output,metadata.user_id,metadata.region,metadata.timestamp,metrics.faithfulness,metrics.relevance,metrics.cost_usd,final_output.fr,final_output.es,final_output.it,project,version
0,trace_001,Who won the 2018 World Cup?,"[{'name': 'retrieval', 'latency_ms': 150, 'out...",France (2018 FIFA World Cup winner),u123,EU,2025-09-24T20:15:00Z,1.00,0.97,0.0025,NaN,NaN,NaN,LLM Evaluation Exercise,1.0
1,trace_002,Explain quantum entanglement simply.,"[{'name': 'prompt_construction', 'latency_ms':...",Quantum entanglement means two particles behav...,u456,US,2025-09-24T20:16:00Z,0.92,0.95,0.0038,NaN,NaN,NaN,LLM Evaluation Exercise,1.0
2,trace_003,"Translate 'Hello World' into French, Spanish, ...","[{'name': 'llm_generation', 'model': 'gpt-4', ...",NaN,u789,APAC,2025-09-24T20:18:00Z,1.00,1.00,0.0012,Bonjour le monde,Hola mundo,Ciao mondo,LLM Evaluation Exercise,1.0


In [50]:

# 1) Keep your original trace-level df
df_steps = df.copy()

# 2) Coerce 'steps' to list (handles None or single dict)
df_steps["steps"] = df_steps["steps"].apply(
    lambda x: [] if x is None
    else ([x] if isinstance(x, dict) else x)
)

# 3) One row per step
steps_long = (
    df_steps
      .explode("steps")
      .dropna(subset=["steps"])
      .reset_index(drop=True)      # <-- important for concat alignment
)

df_steps.head()

,trace_id,user_query,steps,final_output,metadata.user_id,metadata.region,metadata.timestamp,metrics.faithfulness,metrics.relevance,metrics.cost_usd,final_output.fr,final_output.es,final_output.it,project,version
0,trace_001,Who won the 2018 World Cup?,"[{'name': 'retrieval', 'latency_ms': 150, 'out...",France (2018 FIFA World Cup winner),u123,EU,2025-09-24T20:15:00Z,1.00,0.97,0.0025,NaN,NaN,NaN,LLM Evaluation Exercise,1.0
1,trace_002,Explain quantum entanglement simply.,"[{'name': 'prompt_construction', 'latency_ms':...",Quantum entanglement means two particles behav...,u456,US,2025-09-24T20:16:00Z,0.92,0.95,0.0038,NaN,NaN,NaN,LLM Evaluation Exercise,1.0
2,trace_003,"Translate 'Hello World' into French, Spanish, ...","[{'name': 'llm_generation', 'model': 'gpt-4', ...",NaN,u789,APAC,2025-09-24T20:18:00Z,1.00,1.00,0.0012,Bonjour le monde,Hola mundo,Ciao mondo,LLM Evaluation Exercise,1.0


In [51]:
import pandas as pd

# 1) Keep your original trace-level df
df_steps = df.copy()

# 2) Coerce 'steps' to list (handles None or single dict)
df_steps["steps"] = df_steps["steps"].apply(
    lambda x: [] if x is None
    else ([x] if isinstance(x, dict) else x)
)

# 3) One row per step
steps_long = (
    df_steps
      .explode("steps")
      .dropna(subset=["steps"])
      .reset_index(drop=True)      # <-- important for concat alignment
)

# (optional) If some entries aren't dicts, keep only dicts
steps_long = steps_long[steps_long["steps"].apply(lambda x: isinstance(x, dict))].reset_index(drop=True)

# 4) Normalize the dict column and concat
step_cols = pd.json_normalize(steps_long["steps"], sep=".")
steps_long = pd.concat(
    [steps_long.drop(columns="steps").reset_index(drop=True), step_cols],
    axis=1
)

steps_long.head()

,trace_id,user_query,final_output,metadata.user_id,metadata.region,metadata.timestamp,metrics.faithfulness,metrics.relevance,metrics.cost_usd,final_output.fr,...,output_docs,model,temperature,output,token_usage.prompt_tokens,token_usage.completion_tokens,input,output.fr,output.es,output.it
0,trace_001,Who won the 2018 World Cup?,France (2018 FIFA World Cup winner),u123,EU,2025-09-24T20:15:00Z,1.00,0.97,0.0025,NaN,...,"[{'id': 'doc1', 'title': '2018 FIFA Final', 'c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,trace_001,Who won the 2018 World Cup?,France (2018 FIFA World Cup winner),u123,EU,2025-09-24T20:15:00Z,1.00,0.97,0.0025,NaN,...,NaN,gpt-4,0.2,"France won the 2018 FIFA World Cup, beating Cr...",60.0,25.0,NaN,NaN,NaN,NaN
2,trace_002,Explain quantum entanglement simply.,Quantum entanglement means two particles behav...,u456,US,2025-09-24T20:16:00Z,0.92,0.95,0.0038,NaN,...,NaN,NaN,NaN,Explain in simple words: quantum entanglement...,NaN,NaN,Explain quantum entanglement simply.,NaN,NaN,NaN
3,trace_002,Explain quantum entanglement simply.,Quantum entanglement means two particles behav...,u456,US,2025-09-24T20:16:00Z,0.92,0.95,0.0038,NaN,...,NaN,gpt-4,0.7,Quantum entanglement is like having two dice t...,45.0,60.0,NaN,NaN,NaN,NaN
4,trace_003,"Translate 'Hello World' into French, Spanish, ...",NaN,u789,APAC,2025-09-24T20:18:00Z,1.00,1.00,0.0012,Bonjour le monde,...,NaN,gpt-4,0.0,NaN,30.0,15.0,NaN,Bonjour le monde,Hola mundo,Ciao mondo


In [46]:
steps_long.columns

Index(['trace_id', 'user_query', 'final_output', 'metadata.user_id',
       'metadata.region', 'metadata.timestamp', 'metrics.faithfulness',
       'metrics.relevance', 'metrics.cost_usd', 'final_output.fr',
       'final_output.es', 'final_output.it', 'project', 'version', 'name',
       'latency_ms', 'output_docs', 'model', 'temperature', 'output',
       'token_usage.prompt_tokens', 'token_usage.completion_tokens', 'input',
       'output.fr', 'output.es', 'output.it'],
      dtype='object')